In [1]:
import numpy as np
import pandas as pd
import os
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
font_name = fm.FontProperties(fname="C:/Windows/Fonts/malgun.ttf").get_name()
plt.rc("font", family=font_name)
import matplotlib as mlp
mlp.rcParams["axes.unicode_minus"] = False
import seaborn as sns
plt.style.use("fivethirtyeight")
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split

In [2]:
from collections import Counter

# 1. 전처리

## (1) 유튜브 댓글 정리
    - 이모티콘 및 줄바꿈 정리
    - 불용어 정리

In [3]:
# 불용어 
stopwords = pd.read_table("stopwords.txt", encoding='utf-8-sig', header=None)
stopwords = list(stopwords[0])

In [4]:
# 경로 설정
paths = os.listdir("C:/netsong7/pythonwork/AI/raw data(final)/youtubecomment")
paths

['M1유튜브댓글.csv',
 '갤럭시기어유튜브댓글.csv',
 '갤럭시북유튜브댓글.csv',
 '갤럭시유튜브댓글.csv',
 '갤탭유튜브댓글.csv',
 '맥북유튜브댓글.csv',
 '미밴드유튜브댓글.csv',
 '버즈라이브유튜브댓글.csv',
 '버즈플러스유튜브댓글.csv',
 '샤오미워치유튜브댓글.csv',
 '샤오미폰유튜브댓글.csv',
 '샤오미휴대폰유튜브댓글.csv',
 '아이패드유튜브댓글.csv',
 '아이폰유튜브댓글.csv',
 '애플워치유튜브댓글.csv',
 '어메이즈핏유튜브댓글.csv',
 '에어팟유튜브댓글.csv',
 '홍미노트유튜브댓글.csv']

In [5]:
df = pd.DataFrame(columns=["review","keyword"])
df
for path in paths:
    df_1 = pd.read_csv("C:/netsong7/pythonwork/AI/raw data(final)/youtubecomment/"+path ,encoding='utf-8-sig',index_col=0)
    list_review = []
    for i in range(len(df_1)):
        list_review = list_review + list(df_1.loc[i].str.strip().str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]",""))
    df_review = pd.DataFrame(list_review)
    df_review[0] = df_review[0].str.strip()
    df_review["keyword"] = [path[:-9] for i in range(len(df_review))]
    df_review.columns = ["review","keyword"]
    df = pd.concat([df, df_review])
# 스페이스, 탭, nan는 엑셀에서 작업
df.to_csv('youtubecomment.csv',encoding='utf-8-sig' )

In [6]:
df_youtube = pd.read_csv("C:/netsong7/pythonwork/AI/data cleaning(final)/youtubecomment.csv", encoding='utf-8-sig')
df_youtube

,review,keyword
0,끝없는 맥북 논란과 이슈이번에는 화면만 열어도 깨지는디스플레이가 도마 위에 올랐습니다,M1
1,맥북 프로 인치가 저렇게 된 적 있는데 이건 만의 문제가 아니라 맥북 의 문제인 것...,M1
2,개인적으로 애플은 내구성에 대해 조금더 발전된 모습이 필요해 보이는거 같음 아무리 ...,M1
3,고압적인 정책 강제적인 업그레이드 정책 제한된 자유도 이 가지때문에 전 애플로 못...,M1
4,이 영상을 맥북으로 보는 심정이 참 ㅜㅜ,M1
...,...,...
58259,오늘 사전예약한거옴 갤울트라쓰다가홍미잠깐쓰니 적응안댐ㅜ,홍미노트
58260,세상에서 가장 성의없는 리뷰 기자라는 사람들이 제일 공부를 안해 기본스팩만 소개할...,홍미노트
58261,가성비킹임,홍미노트
58262,출고가만원이면 해상도면괜찮지 삼성도 해상도는 만원이상의폰에 넣어 주더라는 만원이...,홍미노트


## (2) 네이버 쇼핑
    - 이모티콘 및 줄바꿈 정리
    - 불용어 정리

In [7]:
# 경로 설정
paths = os.listdir("C:/netsong7/pythonwork/AI/raw data(final)/navershopping")
paths

['삼성 갤럭시 워치 3 41 mm.csv',
 '삼성전자 갤럭시 A32 SM-A325N 64GB [자급제].csv',
 '삼성전자 갤럭시 S20 FE 5G 128GB [자급제].csv',
 '삼성전자 갤럭시 버즈 라이브.csv',
 '삼성전자 갤럭시 버즈 프로.csv',
 '삼성전자 갤럭시 버즈 플러스.csv',
 '삼성전자 갤럭시S21 5G 256GB [자급제].csv',
 '삼성전자 갤럭시노트20울트라 5G 256GB SM-N986N (자급제).csv',
 '삼성전자 갤럭시북 S NT767XCM-K38.csv',
 '삼성전자 갤럭시북 프로360 NT950QDY-A51A.csv',
 '삼성전자 갤럭시북 플렉스 NT950QCG-X716A.csv',
 '삼성전자 삼성 SM-T975 갤럭시 탭 S7+ LTE 256GB.csv',
 '삼성전자 삼성 갤럭시 워치 액티브2 40mm 알루미늄.csv',
 '삼성전자 삼성 갤럭시탭 A7 SM-T500N WIFI 64GB.csv',
 '삼성전자 삼성 갤럭시탭S7FE 12.4 SM-T735 LTE 64GB.csv']

In [8]:
# 스페이스, 탭, nan는 엑셀에서 작업
df = pd.DataFrame(columns=["review","keyword","star"])
for path in paths:
    df_1 = pd.read_csv("C:/netsong7/pythonwork/AI/raw data(final)/navershopping/"+path ,encoding='utf-8-sig',index_col=0)
    df_1["review"] = df_1["review"].str.strip()
    df_1["review"] = df_1["review"].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
    df_1.insert(1, "keyword", [path[:-4] for i in range(len(df_1))])
    df = pd.concat([df, df_1])
df.to_csv("navershopping.csv", encoding='utf-8-sig')


In [9]:
df_shopping = pd.read_csv("C:/netsong7/pythonwork/AI/data cleaning(final)/navershopping.csv", encoding='utf-8-sig')
df_shopping

,review,keyword,star
0,말그래도 최고입니다당일배송 맞구요 택배 파업으로 늦어질 줄 알았는데 구매한 다음날 ...,삼성 갤럭시 워치 3 41 mm,5
1,삼성전자 갤럭시워치 사고는 싶지만 가격대가 높아서 망설이던중 특가하라를 통해서 사고...,삼성 갤럭시 워치 3 41 mm,5
2,실구매가 원에 구매했습니다친구 선물로 사줬는데 엄청 좋아하네요ㅋㅋㅋ손목얇은 남자라 ...,삼성 갤럭시 워치 3 41 mm,5
3,제 생일을 기념해서 부모님께 각각 왓치 하나씩 사드렸어요 아빠는 검은색 엄마는 ...,삼성 갤럭시 워치 3 41 mm,5
4,화면 큰걸로하고싶어서 실버로 구매했는데 손목대비 살짝 큰 감은 있지만 그대로의 멋...,삼성 갤럭시 워치 3 41 mm,5
...,...,...,...
13433,제품 좋아요 잘 사용하고 있습니다,삼성전자 삼성 갤럭시탭S7FE 12.4 SM-T735 LTE 64GB,3
13434,삼성,삼성전자 삼성 갤럭시탭S7FE 12.4 SM-T735 LTE 64GB,3
13435,개인정보만 털린게 아닌지어쨌든 기대해봅니다,삼성전자 삼성 갤럭시탭S7FE 12.4 SM-T735 LTE 64GB,2
13436,구매확정하래ㅋㄲㅋ자증시난다,삼성전자 삼성 갤럭시탭S7FE 12.4 SM-T735 LTE 64GB,1


## (3) 네이버 카페(샤오미)
    - 이모티콘 및 줄바꿈 정리
    - 불용어 정리

In [10]:
# 경로 설정
paths = os.listdir("C:/netsong7/pythonwork/AI/raw data(final)/navercafe")
paths

['샤오미네이버카페리뷰글(스마트폰).csv', '샤오미네이버카페리뷰글(워치).csv']

In [11]:
df = pd.DataFrame(columns= ["title", "review", "keyword" ,"date"])
for path in paths:
    df_1 = pd.read_csv("C:/netsong7/pythonwork/AI/raw data(final)/navercafe/"+path ,encoding='utf-8-sig',index_col=0)
    df_1.insert(2, "keyword", [path[12:-5] for i in range(len(df_1))])
    df_1.columns = ["title", "review", "keyword" ,"date"]
    df_1["review"] = df_1["review"].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
    df_1["review"] = df_1["review"].str.replace("스팸 분쟁이 발견되면 누구라도 바로 신고해주세요    비밀신고   시간접수","")
    df = pd.concat([df, df_1])
df.to_csv("navercafe.csv", encoding='utf-8-sig')

In [12]:
df_cafe = pd.read_csv("C:/netsong7/pythonwork/AI/data cleaning(final)/navercafe.csv", encoding='utf-8-sig')
df_cafe

,title,review,keyword,date
0,홍미노트 시리즈의 고질병 2,홍미노트 프로 프로 프로 프로까지 쭉 써오고 있는데사용 중에 가장 불편하고 짜증...,스마트폰,2021.08.11. 03:56
1,홍미노트 틴 츠로(10 프로) 터치후기,내손이 똥손인거 이 폰이 똥 폰인가내 손이 똥손일까요 이 폰이 똥폰일까요아무리써...,스마트폰,2021.08.11. 01:33
2,미10라에서 미10라줌 롬으로 교체 사용기,안녕하세요미라 단말기에 미라줌 롬 교체기 입니다교체 이유는 주간롬 사용 및 베터...,스마트폰,2021.08.11. 00:44
3,아이폰에서 홍미노트 10 바꾼 후 간단한 일주일 후기입니다.,안녕하세요 기존에 아이폰 미니를 쓰다가안드로이드도 써보고 싶고 샤오미도 궁금하고...,스마트폰,2021.08.07. 20:37
4,poco m3 pro 5g 일주일 사용기 (수정),어느덧 포코 프로 를 받고 사용한지 일주일이 다되가네요이전에 간단하게 짧은 후...,스마트폰,2021.08.07. 11:57
...,...,...,...,...
595,[체험단]Zepp E Square 개봉기 & 초기 설정,이 글은 공식 체험단으로 선정되어 제품을 지급받아 작성했습니다아이나비에서 한국 ...,워치,2020.09.26. 16:48
596,체험단 - ZEPP E SQUARE/43mm 수면 하루지만 확인 해보니,공식 체험단으로 선정 되어 제품을 지급 받아 작성 했습니다 수면 측정이 타 밴...,워치,2020.09.26. 12:46
597,체험단1- ZEPP E SQUARE/43mm 개봉및 설정 메뉴 보기,공식 체험단으로 선정 되어 제품을 지급 받아 작성 했습니다 체험단 수령...,워치,2020.09.26. 02:09
598,알리 발 미밴드5 스트랩 도착,조금 더 비싸길래 그냥 정품 샀는데별 차이 없어보임잘 써야겠어요,워치,2020.09.25. 20:41
